# Kuantisasi dan verifikasi model

In [7]:
import tensorflow as tf
import numpy as np
import h5py
import matplotlib.pyplot as plt
from matplotlib.image import imread
import math
from numpy import genfromtxt

from tensorflow.keras.models import load_model,model_from_json
from tensorflow.keras.utils import CustomObjectScope
from tensorflow.keras.initializers import glorot_uniform


# load model dari file h5

In [8]:
# Model reconstruction from JSON file
with open('model.json', 'r') as f:
    model = model_from_json(f.read())

# Load weights into the new model
model.load_weights('CNN_Model.h5')
model.compile(loss="mse", optimizer="adam")
with h5py.File('CNN_Model.h5','r') as hdf:
    layer_1_data = np.array(hdf.get('layer_1/layer_1/kernel:0'))
    layer_2_data = np.array(hdf.get('layer_2/layer_2/kernel:0'))
    layer_3_data = np.array(hdf.get('layer_3/layer_3/kernel:0'))
    layer_out_data = np.array(hdf.get('layer_Output/layer_Output/kernel:0'))
    
#load dataset
mnist = tf.keras.datasets.mnist
(x_train, y_train),(x_test, y_test) = mnist.load_data()

# Fungsi - fungsi CNN

In [9]:
def pool2d(mat,ksize,method='max',pad=False):
    m, n = mat.shape[1:]
    ky,kx=ksize

    _ceil=lambda x,y: int(numpy.ceil(x/float(y)))

    if pad:
        ny=_ceil(m,ky)
        nx=_ceil(n,kx)
        size=(mat.shape[0],ny*ky, nx*kx)
        mat_pad=numpy.full(size,numpy.nan)
        mat_pad[:m,:n,...]=mat
    else:
        ny=m//ky
        nx=n//kx
        mat_pad=mat[..., :ny*ky, :nx*kx]

    new_shape=(mat.shape[0],ny,ky,nx,kx)

    if method=='max':
        result=np.nanmax(mat_pad.reshape(new_shape),axis=(2,4))
    else:
        result=np.nanmean(mat_pad.reshape(new_shape),axis=(2,4))

    return result

def channel_im2col(A):
    B = [3,3]
    skip=[1,1]
    # Parameters 
    D,M,N = A.shape
    col_extent = N - B[1] + 1
    row_extent = M - B[0] + 1

    # Get Starting block indices
    start_idx = np.arange(B[0])[:,None]*N + np.arange(B[1])

    # Generate Depth indeces
    didx=M*N*np.arange(D)
    start_idx=(didx[:,None]+start_idx.ravel()).reshape((-1,B[0],B[1]))

    # Get offsetted indices across the height and width of input array
    offset_idx = np.arange(row_extent)[:,None]*N + np.arange(col_extent)

    # Get all actual indices & index into input array for final output
    out = np.take (A,start_idx.ravel()[:,None] + offset_idx[::skip[0],::skip[1]].ravel())
    return out


def conv2d(filt,act):
    '''
    conv 2d function
    act = widht, height,depth (w1,h1,d1)
    default = 3,3 filter, stride = 1, p = 0
    im2col reshape
    
    W2=(W1−F+2P)/S+1
    H2=(H1−F+2P)/S+1
    D2=K
    '''
    P = 0
    S = 1
    D1,H1,W1 = act.shape
    n_f,d_f,h_f,w_f = filt.shape
    window_size = d_f*h_f*w_f
    filter_reshape = n_f,window_size
    W2=(W1-h_f +2*P)//S+1
    H2=(H1- h_f+2*P)//S+1
    D2= n_f
    m = channel_im2col(act)
    filt = filt.reshape(filter_reshape)
    res = np.matmul(filt, m)
    res = res.reshape(D2,W2,H2)
    return res

def softmax(X):
    expo = np.exp(X)
    expo_sum = np.sum(np.exp(X))
    return expo/expo_sum

def relu(X):
    return np.maximum(0,X)

def prequant(data,bits):
    MaxValue = np.amax(data)
    MinValue = np.amin(data)
    Range_Real = MaxValue - MinValue
    if(Range_Real==0):
        Range_Real = 1
    Scale = (Range_Real/(pow(2,bits)-1))
    return Scale, Range_Real

def quantize(Val, Range_Real, Scale, zero_point):  
    temp = round(Val * (1/Scale) + zero_point)
    return temp
quantizefunc = np.vectorize(quantize)

In [10]:
layer_1_data = layer_1_data.transpose(3,2,0,1)
layer_2_data= layer_2_data.transpose(3,2,0,1)

# Model Terkuantisasi

In [11]:
def Model2(img):
    Scale_w1,Range_Real_w1 = prequant(layer_1_data,7)
    Scale_d1,Range_Real_d1 = prequant(img.reshape(1,28,28),7)
    quantized_w1 = quantizefunc(layer_1_data,Range_Real_w1,Scale_w1,0)
    quantized_d1 = quantizefunc(img.reshape(1,28,28),Range_Real_d1,Scale_d1,0)
    
    layer_1_out = relu(conv2d(quantized_w1,quantized_d1))
    layer_1_out = Scale_d1*Scale_w1*pool2d(layer_1_out, (2, 2))
    
    Scale_w2,Range_Real_w2 = prequant(layer_2_data,7)
    Scale_d2,Range_Real_d2 = prequant(layer_1_out,7)
    quantized_w2 = quantizefunc(layer_2_data,Range_Real_w2,Scale_w2,0)
    quantized_d2 = quantizefunc(layer_1_out,Range_Real_d2,Scale_d2,0)
    
    layer_2_out = relu(conv2d(quantized_w2,quantized_d2))
    layer_2_out = Scale_d2*Scale_w2*pool2d(layer_2_out, (2, 2))
    layer_2_out = layer_2_out.transpose(1,2,0) ### ubah format data dari CHW ke WHC
    layer_2_out = layer_2_out.reshape(16*5*5)
    
    Scale_w3,Range_Real_w3 = prequant(layer_3_data,7)
    Scale_d3,Range_Real_d3 = prequant(layer_2_out,7)
    quantized_w3 = quantizefunc(layer_3_data,Range_Real_w3,Scale_w3,0)
    quantized_d3 = quantizefunc(layer_2_out,Range_Real_d3,Scale_d3,0)
    
    layer_3_out = np.matmul(quantized_d3,quantized_w3)
    layer_3_out = Scale_d3*Scale_w3*relu(layer_3_out)
    
    Scale_w4,Range_Real_w4 = prequant(layer_out_data,7)
    Scale_d4,Range_Real_d4 = prequant(layer_3_out,7)
    quantized_w4 = quantizefunc(layer_out_data,Range_Real_w4,Scale_w4,0)
    quantized_d4 = quantizefunc(layer_3_out,Range_Real_d4,Scale_d4,0)
    
    layer_last_out = Scale_d4*Scale_w4*np.matmul(quantized_d4,quantized_w4)
    layer_last_out = softmax(layer_last_out)
    return layer_last_out,
    
    

# Verifikasi Kuantisasi

Di code ini model yang terkuantisasi dibandingkan akurasinya dengan model yang tidak terkuantisasi berdasarkan akurasi dengan dataset.


In [ ]:

val_num_uq = 0
val_num_q = 0
correct_q = np.empty(10000)
correct_uq = np.empty(10000)
val_num_q_2 = 0
correct_q_2 = np.empty(10000)

for i in range(10000):
    indata = x_test[i] /255
    label = y_test[i]
    
    #data = indata.ravel()
    
    keras_res = model.predict_classes(indata.reshape(1,28,28,1))
    res = Model2(indata.reshape(1,28,28))
    
    if(label==keras_res[0]):        
        correct_q[val_num_q] = i
        val_num_q += 1

    if(label==np.argmax(res)):
        correct_uq[val_num_uq] = i
        val_num_uq += 1
        
print('val_num_keras_model :', val_num_q)
print('val_num_quantized_model :', val_num_uq)

print('val val_num_keras_model :', (val_num_q/10000)*100,'%')
print('val quantized  :', (val_num_uq/10000)*100,'%')


# Melihat / mengambil weights

In [ ]:
layer_1_data.shape #16 filter, 1 filter per channel, size 3x3

In [ ]:
layer_1_data[0][0] #weight filter pertama